# 4. LangChain Memory - Gestión de Contexto Conversacional

## Objetivos de Aprendizaje
- Comprender la importancia de la memoria en conversaciones con LLMs
- Implementar diferentes tipos de memoria con LangChain
- Gestionar el contexto de conversaciones largas
- Optimizar el uso de tokens con estrategias de memoria

## ¿Por qué es Importante la Memoria?

Los LLMs son **stateless** por naturaleza: no recuerdan conversaciones anteriores. La memoria permite:
- **Contexto conversacional**: Referirse a mensajes anteriores
- **Personalización**: Recordar preferencias del usuario
- **Continuidad**: Mantener hilos de conversación coherentes
- **Experiencia natural**: Conversaciones que se sienten humanas

## Tipos de Memoria en LangChain

1. **ConversationBufferMemory**: Mantiene todo el historial
2. **ConversationSummaryMemory**: Resume conversaciones largas
3. **ConversationBufferWindowMemory**: Mantiene solo los N mensajes más recientes
4. **ConversationSummaryBufferMemory**: Combina resumen + buffer reciente

In [ ]:
# Importar bibliotecas necesarias para memoria
from langchain_openai import ChatOpenAI
from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryMemory,
    ConversationBufferWindowMemory
)
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

import os

print("✓ Bibliotecas de memoria importadas correctamente")

In [ ]:
# Configuración del modelo para memoria
try:
    llm = ChatOpenAI(
        base_url=os.getenv("OPENAI_BASE_URL"),
        api_key=os.getenv("GITHUB_TOKEN"),
        model="gpt-4o-mini",
        temperature=0.1
    )
    
    print("✓ Modelo configurado para experimentos de memoria")
    print(f"Modelo: {llm.model_name}")
    
except Exception as e:
    print(f"✗ Error en configuración: {e}")
    print("Verifica las variables de entorno")

In [ ]:
# Prompt con historial + entrada del usuario
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente útil."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

# Cadena = prompt + modelo
chain = prompt | llm

# Almacén de historiales
store = {}
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Envolver con memoria
conversation = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

## 1. ConversationBufferMemory - Memoria Completa

Esta memoria mantiene **todo** el historial de la conversación. Es la más simple pero puede consumir muchos tokens.

In [ ]:
# Ejemplo básico con RunnableWithMessageHistory

# Prompt con historial + entrada
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente útil."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

# Cadena = prompt + modelo
chain = prompt | llm

# Almacén de memorias por sesión
store = {}

def get_session_history(session_id: str):
    """Devuelve (o crea) el historial completo para la sesión."""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Envolver con RunnableWithMessageHistory
conversation = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

def ejemplo_buffer_memory():
    print("=== CONVERSATIONBUFFERMEMORY ===")
    print("Mantiene todo el historial de conversación\n")
    
    session_id = "demo_session"

    try:
        # Primera interacción
        print("1. Primera pregunta:")
        response1 = conversation.invoke(
            {"input": "Mi nombre es Ana y soy programadora Python"},
            config={"configurable": {"session_id": session_id}}
        )
        print(f"Respuesta: {response1.content}\n")

        # Segunda interacción
        print("2. Segunda pregunta:")
        response2 = conversation.invoke(
            {"input": "¿Cuál es mi nombre y profesión?"},
            config={"configurable": {"session_id": session_id}}
        )
        print(f"Respuesta: {response2.content}\n")

        # Tercera interacción
        print("3. Tercera pregunta:")
        response3 = conversation.invoke(
            {"input": "¿Qué lenguaje de programación mencioné?"},
            config={"configurable": {"session_id": session_id}}
        )
        print(f"Respuesta: {response3.content}\n")

        # Mostrar historial
        print("=== CONTENIDO DE LA MEMORIA ===")
        history = store[session_id].messages
        for i, msg in enumerate(history, 1):
            print(f"{i}. {msg.type}: {msg.content}")

    except Exception as e:
        print(f"Error: {e}")

# Ejecutar
ejemplo_buffer_memory()


## 2. ConversationBufferWindowMemory - Ventana Deslizante

Esta memoria mantiene solo los **N mensajes más recientes**, útil para controlar el uso de tokens.

In [ ]:
# Prompt con historial + entrada
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente útil."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

# Cadena = prompt + modelo
chain = prompt | llm

# Almacén de memorias por sesión
store = {}

class WindowChatMessageHistory(BaseChatMessageHistory):
    """Historial de chat que mantiene solo los últimos k intercambios."""
    
    def __init__(self, k: int = 2):
        self.k = k
        self._messages = []
    
    @property
    def messages(self):
        # Mantener solo los últimos k intercambios (k*2 mensajes: user + assistant)
        return self._messages[-(self.k * 2):]
    
    def add_message(self, message):
        self._messages.append(message)
    
    def clear(self):
        self._messages.clear()

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """Devuelve el historial de ventana para la sesión."""
    if session_id not in store:
        store[session_id] = WindowChatMessageHistory(k=2)
    return store[session_id]

# Envolver con RunnableWithMessageHistory
conversation = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

# Ejemplo
def ejemplo_window_memory():
    print("=== CONVERSATION BUFFER WINDOW MEMORY (k=2) ===")
    print("Mantiene solo los 2 intercambios más recientes\n")
    
    session_id = "demo_window"
    inputs = [
        "Mi nombre es Carlos y tengo 30 años",
        "Trabajo como diseñador gráfico", 
        "Me gusta el café y la música jazz",
        "¿Puedes recordar mi edad?",
        "¿Cuál es mi profesión?"
    ]
    
    try:
        for i, user_input in enumerate(inputs, 1):
            print(f"{'='*20} INTERACCIÓN {i} {'='*20}")
            print(f"👤 Usuario: {user_input}")
            
            response = conversation.invoke(
                {"input": user_input},
                config={"configurable": {"session_id": session_id}}
            )
            print(f"🤖 Asistente: {response.content}\n")
            
            # Obtener el historial
            history = get_session_history(session_id)
            
            # Mostrar comparación clara
            total_messages = len(history._messages)
            visible_messages = len(history.messages)
            
            print(f"📊 ESTADO DE LA MEMORIA:")
            print(f"   💾 Total almacenado: {total_messages} mensajes")
            print(f"   👁️  Visible al modelo: {visible_messages} mensajes")
            print(f"   🗑️  Mensajes descartados: {total_messages - visible_messages}")
            
            # Mensajes almacenados totalmente
            print(f"\n📚 HISTORIAL COMPLETO ALMACENADO ({total_messages} mensajes):")
            if total_messages == 0:
                print("     (Ningún mensaje aún)")
            else:
                for j, msg in enumerate(history._messages, 1):
                    role = "👤 Usuario" if msg.type == "human" else "🤖 Asistente"
                    content = msg.content[:60] + "..." if len(msg.content) > 60 else msg.content
                    # Marcar si está en la ventana visible
                    is_visible = j > total_messages - visible_messages
                    marker = "✅" if is_visible else "❌"
                    print(f"     {j}. {marker} {role}: {content}")
            
            # Lo que ve el modelo
            print(f"\n🔍 VENTANA VISIBLE AL MODELO ({visible_messages} mensajes):")
            if visible_messages == 0:
                print("     (Ningún mensaje visible)")
            else:
                for j, msg in enumerate(history.messages, 1):
                    role = "👤 Usuario" if msg.type == "human" else "🤖 Asistente"
                    content = msg.content[:60] + "..." if len(msg.content) > 60 else msg.content
                    print(f"     {j}. ✅ {role}: {content}")
            
            print("\n" + "="*60 + "\n")
            
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar
ejemplo_window_memory()

## 3. ConversationSummaryMemory - Resumen Inteligente

Esta memoria **resume** conversaciones largas en lugar de mantener todo el texto completo, ahorrando tokens significativamente.

In [ ]:

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Función para resumir automáticamente cuando hay muchos mensajes
def auto_summarize(session_id: str, max_messages=6):
    history = get_session_history(session_id)
    
    if len(history.messages) > max_messages:
        # Mensajes a resumir (todos excepto los últimos 2)
        messages_to_summarize = history.messages[:-2]
        
        # Crear texto para resumir
        conversation_text = ""
        for msg in messages_to_summarize:
            role = "Usuario" if msg.type == "human" else "Asistente"
            conversation_text += f"{role}: {msg.content}\n"
        
        # Generar resumen
        summary_response = llm.invoke(f"Resume esta conversación en 2-3 líneas:\n{conversation_text}")
        summary = summary_response.content
        
        # Reemplazar mensajes antiguos con el resumen
        recent_messages = history.messages[-2:]
        history.clear()
        history.add_ai_message(f"[RESUMEN]: {summary}")
        history.messages.extend(recent_messages)

# Crear conversación
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente útil."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

conversation = RunnableWithMessageHistory(
    prompt | llm,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

def ejemplo_summary_memory():
    print("=== CONVERSATION SUMMARY MEMORY ===")
    print("Resume conversaciones largas para ahorrar tokens\n")
    
    session_id = "summary_session"
    
    # Conversación de ejemplo
    inputs = [
        "Hola, soy María González, ingeniera de software de 35 años",
        "Trabajo en una startup de fintech en Madrid desarrollando pagos digitales",
        "Usamos React, Node.js, Docker y Kubernetes en nuestros proyectos",
        "Mi mayor desafío es la latencia en transacciones internacionales",
        "También trabajo en mejorar la UX de nuestra app móvil",
        "¿Puedes resumir quién soy y cuáles son mis principales desafíos?"
    ]
    
    try:
        for i, user_input in enumerate(inputs, 1):
            print(f"{'='*15} INTERACCIÓN {i} {'='*15}")
            print(f"👤 Usuario: {user_input}")
            
            # Resumir automáticamente si es necesario
            auto_summarize(session_id)
            
            response = conversation.invoke(
                {"input": user_input},
                config={"configurable": {"session_id": session_id}}
            )
            print(f"🤖 Asistente: {response.content}\n")
            
            # Mostrar estado de la memoria
            history = get_session_history(session_id)
            total_messages = len(history.messages)
            
            print(f"📊 ESTADO DE LA MEMORIA:")
            print(f"   💾 Total mensajes: {total_messages}")
            
            # Verificar si hay resumen
            has_summary = any("[RESUMEN]" in msg.content for msg in history.messages if hasattr(msg, 'content'))
            print(f"   📝 Tiene resumen: {'✅ Sí' if has_summary else '❌ No'}")
            
            print(f"\n💬 CONTENIDO ACTUAL DE LA MEMORIA:")
            for j, msg in enumerate(history.messages, 1):
                role = "👤 Usuario" if msg.type == "human" else "🤖 Asistente"
                content = msg.content
                
                # Destacar si es un resumen
                if "[RESUMEN]" in content:
                    role = "📝 Resumen"
                    content = content.replace("[RESUMEN]: ", "")
                
                # Truncar si es muy largo
                if len(content) > 80:
                    content = content[:80] + "..."
                
                print(f"   {j}. {role}: {content}")
            
            print("\n" + "="*50 + "\n")
            
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar
ejemplo_summary_memory()

## Consideraciones Técnicas y Mejores Prácticas

### Selección del Tipo de Memoria

| Tipo | Cuándo Usarlo | Ventajas | Desventajas |
|------|---------------|----------|-------------|
| **Buffer** | Conversaciones cortas | Contexto completo | Alto consumo de tokens |
| **Window** | Contexto reciente importante | Eficiente en tokens | Puede perder información clave |
| **Summary** | Conversaciones muy largas | Balance eficiencia/contexto | Pérdida de detalles específicos |

### Mejores Prácticas:

1. **Gestión de Tokens**:
   - Monitorea el uso de tokens regularmente
   - Establece límites máximos para evitar costos excesivos
   - Considera el costo vs. calidad del contexto

2. **Selección Estratégica**:
   - Usa Buffer para sesiones cortas e importantes
   - Usa Window para conversaciones con contexto limitado
   - Usa Summary para sesiones largas de asistencia

3. **Optimización**:
   - Limpia memoria periódicamente si es necesario
   - Implementa estrategias híbridas según el caso de uso
   - Considera almacenamiento persistente para memoria a largo plazo

## Ejercicios Prácticos

### Ejercicio 1: Análisis de Consumo
Implementa un sistema que monitoree y reporte el uso de tokens con diferentes tipos de memoria.

### Ejercicio 2: Memoria Híbrida
Diseña una estrategia que combine multiple tipos de memoria según el contexto.

### Ejercicio 3: Persistencia
Extiende el chatbot para guardar y cargar memoria entre sesiones.

## Conceptos Clave Aprendidos

1. **Importancia de la memoria** en conversaciones naturales
2. **Tipos de memoria** y sus casos de uso específicos
3. **Balance** entre contexto y eficiencia de tokens
4. **Implementación práctica** con LangChain
5. **Estrategias de optimización** para diferentes escenarios

## Conclusión del Módulo IL1.1

Has completado la introducción a LLMs y conexiones API. Los conceptos aprendidos:

1. **APIs directas** vs **frameworks** como LangChain
2. **Streaming** para mejor experiencia de usuario
3. **Memoria** para conversaciones contextuales
4. **Mejores prácticas** de seguridad y optimización

### Próximos Pasos
En **IL1.2** exploraremos técnicas avanzadas de **prompt engineering** incluyendo zero-shot, few-shot, y chain-of-thought prompting.